In [1]:
import os

In [2]:
%pwd

'/Users/brunostordeur/Documents/GitHub/End-to-End-Deep-Learning-Project/research'

In [3]:
os.chdir("../")
%pwd

'/Users/brunostordeur/Documents/GitHub/End-to-End-Deep-Learning-Project'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url : str
    local_data_file: Path
    unzip_dir: Path



In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, get_size

In [6]:
@dataclass()
class ConfigurationManager:
    def __init__(self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
            )
        return data_ingestion_config


In [7]:
import os 
import zipfile
import urllib.request as request
from cnnClassifier import logger


In [8]:
class DataIngestion:
    def __init__(self,config : DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            try:
                filename, headers = request.urlretrieve(
                    url= self.config.source_url,
                    filename = self.config.local_data_file
                )
                logger.info(f"filename {filename} downloaded! with following info: \n {headers}")
            except Exception as e:
                logger.error(f" failed to download file:{e}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extract the zip file into the data directory
        Function returns None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        print(f"Attempting to open file at {self.config.local_data_file}")  # Add debugging output

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            print("Successfully opened zip file")
            zip_ref.extractall(unzip_path)
            

In [9]:
try: 
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e
    

[2023-07-20 11:59:05,420: INFO: common: yaml file: <_io.TextIOWrapper name='config/config.yaml' mode='r' encoding='UTF-8'> loaded successfuly]
[2023-07-20 11:59:05,422: INFO: common: yaml file: <_io.TextIOWrapper name='params.yaml' mode='r' encoding='UTF-8'> loaded successfuly]
[2023-07-20 11:59:05,422: INFO: common: created directory at: artifacts]
[2023-07-20 11:59:05,423: INFO: common: created directory at: artifacts/data_ingestion]
[2023-07-20 11:59:05,423: INFO: 3996140745: File already exists of size: ~ 11345 KB]
Attempting to open file at artifacts/data_ingestion/data.zip
Successfully opened zip file


'/Users/brunostordeur/Documents/GitHub/End-to-End-Deep-Learning-Project/research'